# ML with TensorFlow Extended (TFX) -- Part 1
The puprpose of this tutorial is to show how to do end-to-end ML with TFX libraries on Google Cloud Platform. This tutorial covers:
1. Data analysis and schema generation with **TF Data Validation**.
2. Data preprocessing with **TF Transform**.
3. Model training with **TF Estimator**.
4. Model evaluation with **TF Model Analysis**.

This notebook has been tested in Jupyter on the Deep Learning VM.

## 0. Setup Python and Cloud environment

Install the libraries we need and set up variables to reference our project and bucket.

In [3]:
%pip install -q --upgrade grpcio_tools tensorflow_data_validation

Note: you may need to restart the kernel to use updated packages.


In [1]:
import apache_beam as beam
import platform
import tensorflow as tf
import tensorflow_data_validation as tfdv
import tensorflow_transform as tft
import tornado

print('tornado version: {}'.format(tornado.version))
print('Python version: {}'.format(platform.python_version()))
print('TF version: {}'.format(tf.__version__))
print('TFT version: {}'.format(tft.__version__))
print('TFDV version: {}'.format(tfdv.__version__))
print('Apache Beam version: {}'.format(beam.__version__))

/home/jupyter/.local/lib/python3.5/site-packages/apache_beam/__init__.py:84: UserWarning: Running the Apache Beam SDK on Python 3 is not yet fully supported. You may encounter buggy behavior or missing features.
  'Running the Apache Beam SDK on Python 3 is not yet fully supported. '


tornado version: 5.1.1
Python version: 3.5.3
TF version: 1.13.1
TFT version: 0.13.0
TFDV version: 0.13.1
Apache Beam version: 2.11.0


In [2]:
PROJECT = 'cloud-training-demos'    # Replace with your PROJECT
BUCKET = 'cloud-training-demos-ml'  # Replace with your BUCKET
REGION = 'us-central1'              # Choose an available region for Cloud MLE

import os

os.environ['PROJECT'] = PROJECT
os.environ['BUCKET'] = BUCKET
os.environ['REGION'] = REGION

## ensure we're using python2 env
os.environ['CLOUDSDK_PYTHON'] = 'python2'

In [3]:
%%bash
gcloud config set project $PROJECT
gcloud config set compute/region $REGION

## ensure we predict locally with our current Python environment
gcloud config set ml_engine/local_python `which python`

Updated property [core/project].
Updated property [compute/region].
Updated property [ml_engine/local_python].


<img valign="middle" src="images/tfx.jpeg">

### UCI Adult Dataset: https://archive.ics.uci.edu/ml/datasets/adult
Predict whether income exceeds $50K/yr based on census data. Also known as "Census Income" dataset.

In [4]:
DATA_DIR='gs://cloud-samples-data/ml-engine/census/data'

In [5]:
import os

TRAIN_DATA_FILE = os.path.join(DATA_DIR, 'adult.data.csv')
EVAL_DATA_FILE = os.path.join(DATA_DIR, 'adult.test.csv')
!gsutil ls -l $TRAIN_DATA_FILE
!gsutil ls -l $EVAL_DATA_FILE

   3974304  2018-05-10T22:21:14Z  gs://cloud-samples-data/ml-engine/census/data/adult.data.csv
TOTAL: 1 objects, 3974304 bytes (3.79 MiB)
   1986465  2018-05-10T22:21:14Z  gs://cloud-samples-data/ml-engine/census/data/adult.test.csv
TOTAL: 1 objects, 1986465 bytes (1.89 MiB)


## 1. Data Analysis
For data analysis, visualization, and schema generation, we use [TensorFlow Data Validation](https://www.tensorflow.org/tfx/guide/tfdv) to perform the following:
1. **Analyze** the training data and produce **statistics**.
2. Generate data **schema** from the produced statistics.
3. **Configure** the schema.
4. **Validate** the evaluation data against the schema.
5. **Save** the schema for later use.

In [6]:
import tensorflow_data_validation as tfdv
print('TFDV version: {}'.format(tfdv.__version__))

TFDV version: 0.13.1


### 1.1 Compute and visualise statistics

In [7]:
HEADER = ['age', 'workclass', 'fnlwgt', 'education', 'education_num',
               'marital_status', 'occupation', 'relationship', 'race', 'gender',
               'capital_gain', 'capital_loss', 'hours_per_week',
               'native_country', 'income_bracket']

TARGET_FEATURE_NAME = 'income_bracket'
TARGET_LABELS = [' <=50K', ' >50K']
WEIGHT_COLUMN_NAME = 'fnlwgt'

In [8]:
train_stats = tfdv.generate_statistics_from_csv(
    data_location=TRAIN_DATA_FILE, 
    column_names=HEADER,
    stats_options=tfdv.StatsOptions(
        weight_feature=WEIGHT_COLUMN_NAME,
        sample_rate=1.0
    )
)

Instructions for updating:
Use eager execution and: 
`tf.data.TFRecordDataset(path)`


Instructions for updating:
Use eager execution and: 
`tf.data.TFRecordDataset(path)`


In [9]:
tfdv.visualize_statistics(train_stats)

### 1.2 Infer Schema

In [10]:
schema = tfdv.infer_schema(statistics=train_stats)
tfdv.display_schema(schema=schema)

,Type,Presence,Valency,Domain
Feature name,,,,
'relationship',STRING,required,,'relationship'
'education',STRING,required,,'education'
'occupation',STRING,required,,'occupation'
'workclass',STRING,required,,'workclass'
'capital_loss',INT,required,,-
'age',INT,required,,-
'hours_per_week',INT,required,,-
'income_bracket',STRING,required,,'income_bracket'
'native_country',STRING,required,,'native_country'


,Values
Domain,
'relationship',"' Husband', ' Not-in-family', ' Other-relative', ' Own-child', ' Unmarried', ' Wife'"
'education',"' 10th', ' 11th', ' 12th', ' 1st-4th', ' 5th-6th', ' 7th-8th', ' 9th', ' Assoc-acdm', ' Assoc-voc', ' Bachelors', ' Doctorate', ' HS-grad', ' Masters', ' Preschool', ' Prof-school', ' Some-college'"
'occupation',"' ?', ' Adm-clerical', ' Armed-Forces', ' Craft-repair', ' Exec-managerial', ' Farming-fishing', ' Handlers-cleaners', ' Machine-op-inspct', ' Other-service', ' Priv-house-serv', ' Prof-specialty', ' Protective-serv', ' Sales', ' Tech-support', ' Transport-moving'"
'workclass',"' ?', ' Federal-gov', ' Local-gov', ' Never-worked', ' Private', ' Self-emp-inc', ' Self-emp-not-inc', ' State-gov', ' Without-pay'"
'income_bracket',"' <=50K', ' >50K'"
'native_country',"' ?', ' Cambodia', ' Canada', ' China', ' Columbia', ' Cuba', ' Dominican-Republic', ' Ecuador', ' El-Salvador', ' England', ' France', ' Germany', ' Greece', ' Guatemala', ' Haiti', ' Holand-Netherlands', ' Honduras', ' Hong', ' Hungary', ' India', ' Iran', ' Ireland', ' Italy', ' Jamaica', ' Japan', ' Laos', ' Mexico', ' Nicaragua', ' Outlying-US(Guam-USVI-etc)', ' Peru', ' Philippines', ' Poland', ' Portugal', ' Puerto-Rico', ' Scotland', ' South', ' Taiwan', ' Thailand', ' Trinadad&Tobago', ' United-States', ' Vietnam', ' Yugoslavia'"
'race',"' Amer-Indian-Eskimo', ' Asian-Pac-Islander', ' Black', ' Other', ' White'"
'marital_status',"' Divorced', ' Married-AF-spouse', ' Married-civ-spouse', ' Married-spouse-absent', ' Never-married', ' Separated', ' Widowed'"
'gender',"' Female', ' Male'"


In [11]:
schema

feature {
  name: "relationship"
  type: BYTES
  domain: "relationship"
  presence {
    min_fraction: 1.0
    min_count: 1
  }
  shape {
    dim {
      size: 1
    }
  }
}
feature {
  name: "education"
  type: BYTES
  domain: "education"
  presence {
    min_fraction: 1.0
    min_count: 1
  }
  shape {
    dim {
      size: 1
    }
  }
}
feature {
  name: "occupation"
  type: BYTES
  domain: "occupation"
  presence {
    min_fraction: 1.0
    min_count: 1
  }
  shape {
    dim {
      size: 1
    }
  }
}
feature {
  name: "workclass"
  type: BYTES
  domain: "workclass"
  presence {
    min_fraction: 1.0
    min_count: 1
  }
  shape {
    dim {
      size: 1
    }
  }
}
feature {
  name: "capital_loss"
  type: INT
  presence {
    min_fraction: 1.0
    min_count: 1
  }
  shape {
    dim {
      size: 1
    }
  }
}
feature {
  name: "age"
  type: INT
  presence {
    min_fraction: 1.0
    min_count: 1
  }
  shape {
    dim {
      size: 1
    }
  }
}
feature {
  name: "hours_per_week"


### 1.3 Configure Schema

In [12]:
# Relax the minimum fraction of values that must come from the domain for feature occupation.
occupation = tfdv.get_feature(schema, 'occupation')
occupation.distribution_constraints.min_domain_mass = 0.9

# Add new value to the domain of feature native_country.
native_country_domain = tfdv.get_domain(schema, 'native_country')
native_country_domain.value.append('Egypt')

# All features are by default in both TRAINING and SERVING environments.
schema.default_environment.append('TRAINING')
schema.default_environment.append('EVALUATION')
schema.default_environment.append('SERVING')

# Specify that weight and class feature is not in SERVING environment.
#tfdv.get_feature(schema, WEIGHT_COLUMN_NAME).not_in_environment.append('SERVING')
#tfdv.get_feature(schema, WEIGHT_COLUMN_NAME).not_in_environment.append('EVALUATION')
#tfdv.get_feature(schema, TARGET_FEATURE_NAME).not_in_environment.append('SERVING')

In [13]:
tfdv.display_schema(schema=schema)

,Type,Presence,Valency,Domain
Feature name,,,,
'relationship',STRING,required,,'relationship'
'education',STRING,required,,'education'
'occupation',STRING,required,,'occupation'
'workclass',STRING,required,,'workclass'
'capital_loss',INT,required,,-
'age',INT,required,,-
'hours_per_week',INT,required,,-
'income_bracket',STRING,required,,'income_bracket'
'native_country',STRING,required,,'native_country'


,Values
Domain,
'relationship',"' Husband', ' Not-in-family', ' Other-relative', ' Own-child', ' Unmarried', ' Wife'"
'education',"' 10th', ' 11th', ' 12th', ' 1st-4th', ' 5th-6th', ' 7th-8th', ' 9th', ' Assoc-acdm', ' Assoc-voc', ' Bachelors', ' Doctorate', ' HS-grad', ' Masters', ' Preschool', ' Prof-school', ' Some-college'"
'occupation',"' ?', ' Adm-clerical', ' Armed-Forces', ' Craft-repair', ' Exec-managerial', ' Farming-fishing', ' Handlers-cleaners', ' Machine-op-inspct', ' Other-service', ' Priv-house-serv', ' Prof-specialty', ' Protective-serv', ' Sales', ' Tech-support', ' Transport-moving'"
'workclass',"' ?', ' Federal-gov', ' Local-gov', ' Never-worked', ' Private', ' Self-emp-inc', ' Self-emp-not-inc', ' State-gov', ' Without-pay'"
'income_bracket',"' <=50K', ' >50K'"
'native_country',"' ?', ' Cambodia', ' Canada', ' China', ' Columbia', ' Cuba', ' Dominican-Republic', ' Ecuador', ' El-Salvador', ' England', ' France', ' Germany', ' Greece', ' Guatemala', ' Haiti', ' Holand-Netherlands', ' Honduras', ' Hong', ' Hungary', ' India', ' Iran', ' Ireland', ' Italy', ' Jamaica', ' Japan', ' Laos', ' Mexico', ' Nicaragua', ' Outlying-US(Guam-USVI-etc)', ' Peru', ' Philippines', ' Poland', ' Portugal', ' Puerto-Rico', ' Scotland', ' South', ' Taiwan', ' Thailand', ' Trinadad&Tobago', ' United-States', ' Vietnam', ' Yugoslavia', 'Egypt'"
'race',"' Amer-Indian-Eskimo', ' Asian-Pac-Islander', ' Black', ' Other', ' White'"
'marital_status',"' Divorced', ' Married-AF-spouse', ' Married-civ-spouse', ' Married-spouse-absent', ' Never-married', ' Separated', ' Widowed'"
'gender',"' Female', ' Male'"


### 1.4 Validate evaluation data

In [14]:
eval_stats = tfdv.generate_statistics_from_csv(EVAL_DATA_FILE, column_names=HEADER)
eval_anomalies = tfdv.validate_statistics(eval_stats, schema, environment='EVALUATION')
tfdv.display_anomalies(eval_anomalies)

,Anomaly short description,Anomaly long description
Feature name,,
'fnlwgt',New column,New column (column in data but not in schema)


### 1.5 Freeze the schema

In [15]:
RAW_SCHEMA_LOCATION = 'raw_schema.pbtxt'

In [16]:
from tensorflow.python.lib.io import file_io
from google.protobuf import text_format

tfdv.write_schema_text(schema, RAW_SCHEMA_LOCATION)

In [17]:
!cat {RAW_SCHEMA_LOCATION}

feature {
  name: "relationship"
  type: BYTES
  domain: "relationship"
  presence {
    min_fraction: 1.0
    min_count: 1
  }
  shape {
    dim {
      size: 1
    }
  }
}
feature {
  name: "education"
  type: BYTES
  domain: "education"
  presence {
    min_fraction: 1.0
    min_count: 1
  }
  shape {
    dim {
      size: 1
    }
  }
}
feature {
  name: "occupation"
  type: BYTES
  domain: "occupation"
  presence {
    min_fraction: 1.0
    min_count: 1
  }
  distribution_constraints {
    min_domain_mass: 0.9
  }
  shape {
    dim {
      size: 1
    }
  }
}
feature {
  name: "workclass"
  type: BYTES
  domain: "workclass"
  presence {
    min_fraction: 1.0
    min_count: 1
  }
  shape {
    dim {
      size: 1
    }
  }
}
feature {
  name: "capital_loss"
  type: INT
  presence {
    min_fraction: 1.0
    min_count: 1
  }
  shape {
    dim {
      size: 1
    }
  }
}
feature {
  name: "age"
  type: INT
  presence {
    min_fraction: 1.0
    min_count: 1
  }
  shape {
    dim {
   

## License

Copyright 2019 Google LLC

Licensed under the Apache License, Version 2.0 (the "License");
you may not use this file except in compliance with the License.
You may obtain a copy of the License at http://www.apache.org/licenses/LICENSE-2.0.

Unless required by applicable law or agreed to in writing, software
distributed under the License is distributed on an "AS IS" BASIS,
WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
See the License for the specific language governing permissions and
limitations under the License.

---
**Disclaimer**: This is not an official Google product. The sample code provided for an educational purpose.

---